<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows - Cloud/Jira_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [2]:
#! pandas Config

import pandas as pd

pd.options.mode.use_inf_as_na = True
pd.set_option("display.max_rows", 10)
pd.set_option("display.expand_frame_repr", True)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 200000
pd.options.display.max_rows = 10

In [12]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    dotenv.load_dotenv('.env')

    Host = os.getenv('SECRETS_HOST') 
    Username = os.getenv('SECRETS_USERNAME') 
    Password = os.getenv('SECRETS_PASSWORD') 
except: 
    pass 

if Host is None or Host == "":
    Host = input("Enter Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Host: " + Host)

ValidProjectCategories = ["'Client Delivery Project'"]
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

In [4]:
#! Functions

import time
import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        

        response = requests.post(Host + url, headers = headers, json = defaultContents)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)
    df = df[["issuetypes"]]
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid.columns.values else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

def fnGetTimeZoneOffset() -> str:
    offset = (time.timezone if (time.localtime().tm_isdst == 0) else time.altzone) / 60 / 60 * -1
    return ("" if(offset) < 0 else "+") + str(int((offset - (offset % 1)))).zfill(2) + ":" + str(int((offset % 1) * 60)).zfill(2)

goldenDF = None
globals()['goldenDF'] = None 

def fnGetGoldenCopy() -> pd.DataFrame:
    if 'goldenDF' not in globals() or goldenDF is None: 
        raise Exception("Base data frame not loaded") 
    else:
        df:pd.DataFrame = goldenDF.copy(deep = True)
        if df is None or len(df.index) == 0:
            raise Exception ("No results")
    return df

def exit():
    class StopExecution(Exception):
        def _render_traceback_(self):
            []
    raise StopExecution


In [5]:
#! Jira Issues Capture

from IPython.display import display

ExpectedIssueTypes = [ "Task" ]
JQL = "issuetype in (" + ','.join(ExpectedIssueTypes) + ") and category in (" + ','.join(ValidProjectCategories) + ") ORDER BY updatedDate DESC"

fields = fnGetIssueTypeFields(ExpectedIssueTypes)

df = fnSearch(JQL, fields["fieldId"].values)
if df is None:	
	display("No results")
	exit()

df = df.drop(["expand", "startAt", "maxResults", "total"], axis=1)
df = df.explode("issues")
df = df.expand("issues", [], None, False)
df = df.drop(["expand", "self"], axis=1)
df = df.expand("fields", [], None, False)		

df = df.rename( columns=dict( zip ( fields.fieldId, fields.name )) )	

df = df.convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')

goldenDF:pd.DataFrame = df.copy(deep = True)
globals()['goldenDF'] = goldenDF # make this globally available 

df:pd.DataFrame = fnGetGoldenCopy()
display(df.dtypes)
display(df.head())

Id                        Int64
Key              string[python]
Issue Type               object
Components               object
Project                  object
                      ...      
Linked Issues            object
Assignee                 object
Updated          string[python]
Status                   object
Parent                   object
Length: 28, dtype: object

In [6]:
#! Issues

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df.drop(["Components", "Attachment", "Linked Issues", "Sprint", "Fix versions", "Labels"], axis=1)
df = df.expand("Issue Type", ["id", "name"])
df = df.expand("Status", ["id", "name", "statusCategory"]).sentence_case_columns()
df = df.expand("Status Category", ["id", "name"]) 
df = df.expand("Project", ["id", "name", "projectCategory"]).sentence_case_columns()
df = df.expand("Project Category", ["id", "name"])
df = df.expand("Reporter", ["accountId", "displayName"])
df = df.expand("Assignee", ["accountId", "displayName"])
df = df.expand("Priority", ["id", "name"])
df = df.expand("Resolution", ["id", "name"])
df = df.expand("Parent", ["id", "key"])     
df = df.expand("Account", ["id", "value"])   
df = df.expand("Tempo Customer", ["id", "value"])   

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Story Points"] = df["Story Points"].astype('float')

display(df.dtypes)
display(df.head())

Id                               Int64
Key                     string[python]
Description             string[python]
Epic Link               string[python]
Resolution Date         string[python]
                             ...      
Parent Key              string[python]
Account Id                       Int64
Account Value           string[python]
Tempo Customer Id       string[python]
Tempo Customer Value    string[python]
Length: 36, dtype: object

In [7]:
#! Components

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Components"]]
df = df[df["Components"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Components")
df = df.expand("Components", ["id", "name"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                          Int64
Key                string[python]
Components Id      string[python]
Components Name    string[python]
dtype: object

In [8]:
#! Linked Issues

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Linked Issues"]]
df = df[df["Linked Issues"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Linked Issues")
df = df.expand("Linked Issues", ["id", "type", "inwardIssue", "outwardIssue"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

display(df.dtypes)
display(df.head())

Id                                      Int64
Key                            string[python]
Linked Issues Id               string[python]
Linked Issues Type                     object
Linked Issues Inward Issue             object
Linked Issues Outward Issue            object
dtype: object

In [9]:
#! Sprints

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Sprint"]]
df = df[~df["Sprint"].isna()]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Sprint")
df = df.expand("Sprint")

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)

df["Sprint Start Date"] = df["Sprint Start Date"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))
df["Sprint End Date"] = df["Sprint End Date"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))
df["Sprint Complete Date"] = df["Sprint Complete Date"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))

display(df.dtypes)
display(df.head())

'No results'

In [10]:
#! Attachments

from IPython.display import display
from pandas.core.dtypes.dtypes import DatetimeTZDtype

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Attachment"]]
df = df[df["Attachment"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Attachment")
df = df.expand("Attachment", ["id", "filename", "created", "mimeType", "size", "thumbnail", "content", "author"])
df = df.expand("Attachment Author", ["accountId"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Attachment Id"] = df["Attachment Id"].astype('Int64')
df["Attachment Size"] = df["Attachment Size"].astype('float64')
df["Attachment Created"] = df["Attachment Created"].astype(DatetimeTZDtype("ns", fnGetTimeZoneOffset()))

display(df.dtypes)
display(df.head())

Id                                                  Int64
Key                                        string[python]
Attachment Id                                       Int64
Attachment Filename                        string[python]
Attachment Created              datetime64[ns, UTC+10:00]
Attachment Mime Type                       string[python]
Attachment Size                                   float64
Attachment Thumbnail                       string[python]
Attachment Content                         string[python]
Attachment Author Account Id               string[python]
dtype: object

In [11]:
#! Versions

from IPython.display import display

df:pd.DataFrame = fnGetGoldenCopy()
df = df[["Id", "Key", "Fix versions"]]
df = df[df["Fix versions"].map(lambda d: len(d)) > 0]

if df is None or len(df.index) == 0:	
    display("No results")
    exit()

df = df.explode("Fix versions")
df = df.expand("Fix versions", ["id", "name"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Fix Versions Id"] = df["Fix Versions Id"].astype('Int64')

display(df.dtypes)
display(df.head())

Id                            Int64
Key                  string[python]
Fix Versions Id               Int64
Fix Versions Name    string[python]
dtype: object